本笔记本使用大庆数据hdf5文件进行标准化处理过程

大庆数据

1. 定义常量和配置

In [19]:
# constants.py

# 列名
COLUMNS = ['DEPT', 'RMN-RMG', 'CAL', 'SP', 'GR', 'HAC', 'BHC', 'DEN']


2. H5文件模块

读取

In [3]:
# data_read.py

import h5py
import pandas as pd

def load_data_from_h5(filename):
    """
    从.h5文件中读取数据，并返回一个Pandas DataFrame。

    Parameters:
    - filename (str): 文件名，包括路径和扩展名。

    Returns:
    - df (DataFrame): 包含从文件中读取的数据的DataFrame。
    """
    with h5py.File(filename, 'r') as h5f:
        # 获取文件的描述信息
        file_description = h5f.attrs.get('file_description', 'No description')
        print(f"File Description: {file_description}")
        
        # 初始化一个空的DataFrame
        df = pd.DataFrame()
        
        # 遍历文件中的所有数据集
        for key in h5f.keys():
            print(f"Reading {key}...")
            # 读取数据集
            dataset = h5f[key]
            # 将数据集添加到DataFrame中
            df[key] = dataset[:]
            # 打印数据集的单位和描述
            unit = dataset.attrs.get('unit', 'No unit')
            description = dataset.attrs.get('description', 'No description')
            print(f"{key}: Unit = {unit}, Description = {description}")
        
        # 在添加数据集到DataFrame之后，将WellName列中的字节字符串转换为字符串
        df['WellName'] = df['WellName'].apply(lambda x: x.decode('utf-8'))
        return df

filename = './well_log_daqing.h5'
df = load_data_from_h5(filename)
print(df.head())

File Description: This HDF5 file contains oil well log data collected from various wells in the North Sea region. The data includes measurements of depth, resistivity, caliper, sound velocity, gamma ray, and density, among others. The measurements were taken using a combination of logging tools and techniques, including electrical resistivity logging (ERL), caliper logging, and gamma ray logging. The data has been processed to ensure consistency and accuracy, including normalization and interpolation where necessary. This file is intended for use in geological and geophysical analyses, particularly in the study of oil and gas reservoirs. For any questions or further information, please contact the data provider at [email protected]
Reading BHC...
BHC: Unit = us/m, Description = 表面声波时差，没有明确单位，可能是微秒每米 (.us/m)，因为与HAC类似
Reading CAL...
CAL: Unit = cm, Description = 孔隙径，单位为厘米 (.cm)
Reading DEN...
DEN: Unit = g/cm3, Description = 密度，单位为克/立方厘米 (.g/cm3)
Reading DEPT...
DEPT: Unit = m, Descripti

3. 数据处理模块

读取数据之后，需要进行标准化出来。

In [20]:
# data_process.py

import pandas as pd
from sklearn.preprocessing import StandardScaler
import joblib
# form constants import COLUMNS

class DataStandardizer:
    """
    数据标准化类，使用StandardScaler进行标准化。
    """
    def __init__(self, columns=None):
        self.scaler = StandardScaler()
        self.columns = columns
        self.fitted = False

    def fit(self, df):
        """
        拟合标准化器。
        """
        if self.columns is None:
            self.columns = df.columns
        self.scaler.fit(df[self.columns])
        self.fitted = True

    def transform(self, df):
        """  
        对数据进行标准化。
        """
        if not self.fitted:
            raise ValueError("Scaler is not fitted yet. Call 'fit' with appropriate data before transforming.")
        # 标准化指定的列
        df_standardized = pd.DataFrame(self.scaler.transform(df[self.columns]), columns=self.columns)
        # 将标准化后的列替换原始数据框中的对应列
        for col in self.columns:
            df[col] = df_standardized[col]
        return df

    def inverse_transform(self, df):
        """ 
        对标准化后的数据进行反标准化。
        """
        if not self.fitted:
            raise ValueError("Scaler is not fitted yet. Call 'fit' with appropriate data before inverse transforming.")
        # 反标准化指定的列
        df_standardized = pd.DataFrame(self.scaler.inverse_transform(df[self.columns]), columns=self.columns)
        # 将反标准化后的列替换原始数据框中的对应列
        for col in self.columns:
            df[col] = df_standardized[col]
        return df

    def save(self, filename):
        """  
        保存标准化器。
        """
        joblib.dump(self.scaler, filename)

    def load(self, filename):
        """ 
        加载标准化器。
        """
        self.scaler = joblib.load(filename)
        self.fitted = True

# 示例使用
standardizer = DataStandardizer(columns= COLUMNS) # 指定需要标准化的列
standardizer.fit(df) # 使用df数据进行标准化
df_standardized = standardizer.transform(df) # 标准化df数据

# 保存标准化器
standardizer.save('scaler.joblib')

# # 加载标准化器
# standardizer.load('scaler.joblib')

# # 反标准化
# df_original = standardizer.inverse_transform(df_standardized)

In [5]:
df_standardized

,BHC,CAL,DEN,DEPT,GR,HAC,RMG,RMN,RMN-RMG,SP,WellName
0,1.354331,1.382265,0.021540,-1.575040,-0.076729,1.203110,2.260,2.265,-0.687548,1.389699,A1
1,1.331241,1.345440,0.073887,-1.574611,-0.067284,1.092743,2.241,2.281,-0.658715,1.389699,A1
2,1.314226,1.298230,0.178583,-1.574182,-0.068630,1.044477,2.405,2.474,-0.634824,1.378332,A1
3,1.325168,1.260461,0.073887,-1.573753,-0.076729,1.090462,2.598,2.640,-0.657067,1.358426,A1
4,1.338521,1.232135,0.178583,-1.573324,-0.094258,1.242187,2.533,2.538,-0.687548,1.338520,A1
...,...,...,...,...,...,...,...,...,...,...,...
38729,-0.231540,-0.852690,-1.580308,0.359415,-0.367605,-0.233752,6.191,8.237,0.993849,-1.949959,A6
38730,-0.282565,-0.881016,-1.287160,0.359844,-0.346479,-0.233752,5.867,8.042,1.100121,-1.953868,A6
38731,-0.332362,-0.895179,-1.077768,0.360273,-0.320068,-0.240638,6.723,9.164,1.319255,-1.959762,A6
38732,-0.373673,-0.859299,-0.931194,0.360702,-0.279134,-0.240638,7.263,9.530,1.175912,-1.953868,A6


4. 数据保存模块

In [6]:
# data_save.py

import h5py
import numpy as np

def save_data_to_h5(data, filename, file_description):
    """
    将数据保存为.h5文件，并添加文件的描述信息作为属性。

    Parameters:
    - data (DataFrame or ndarray): 要保存的数据。
    - filename (str): 文件名，包括路径和扩展名。
    - file_description (str): 整个文件的描述信息。
    """
    with h5py.File(filename, 'w') as h5f:
        # 添加整个文件的描述信息作为根组的属性
        h5f.attrs['file_description'] = file_description
        
        # 创建数据集
        for key in data.columns:
            dataset = h5f.create_dataset(key, data=data[key].values)

In [13]:
import h5py
import numpy as np
import pandas as pd

def load_data_from_h5(filename):
    """
    从.h5文件中读取数据和文件描述信息。

    Parameters:
    - filename (str): 文件名，包括路径和扩展名。

    Returns:
    - data (DataFrame): 从文件中读取的数据。
    - file_description (str): 文件的描述信息。
    """
    with h5py.File(filename, 'r') as h5f:
        # 读取文件描述信息
        file_description = h5f.attrs.get('file_description', 'No description available')

        data = {key: h5f[key][()] for key in h5f.keys()}
        # 将数据转换为DataFrame
        data_df = pd.DataFrame(data)
        
        # 在添加数据集到DataFrame之后，将WellName列中的字节字符串转换为字符串
        data_df['WellName'] = data_df['WellName'].apply(lambda x: x.decode('utf-8'))

        return data_df, file_description



In [7]:
file_description = "将标准化后的数据保存为.h5文件，以便后续使用。标准化器为StandardScaler。"
save_data_to_h5(df_standardized, 'well_log_daqing_standardized.h5', file_description)
print("数据已保存为well_log_daqing_standardized.h5。还有一个标准化器scaler.joblib。使用的标准化器为StandardScaler。")

数据已保存为well_log_daqing_standardized.h5。还有一个标准化器scaler.joblib。使用的标准化器为StandardScaler。


In [14]:
# 示例使用
filename = './well_log_daqing_standardized.h5' # 请替换为您的文件名
data, description = load_data_from_h5(filename)
print("File Description:", description)
print("Data:", data)

File Description: 将标准化后的数据保存为.h5文件，以便后续使用。标准化器为StandardScaler。
Data:             BHC       CAL       DEN      DEPT        GR       HAC    RMG  \
0      1.354331  1.382265  0.021540 -1.575040 -0.076729  1.203110  2.260   
1      1.331241  1.345440  0.073887 -1.574611 -0.067284  1.092743  2.241   
2      1.314226  1.298230  0.178583 -1.574182 -0.068630  1.044477  2.405   
3      1.325168  1.260461  0.073887 -1.573753 -0.076729  1.090462  2.598   
4      1.338521  1.232135  0.178583 -1.573324 -0.094258  1.242187  2.533   
...         ...       ...       ...       ...       ...       ...    ...   
38729 -0.231540 -0.852690 -1.580308  0.359415 -0.367605 -0.233752  6.191   
38730 -0.282565 -0.881016 -1.287160  0.359844 -0.346479 -0.233752  5.867   
38731 -0.332362 -0.895179 -1.077768  0.360273 -0.320068 -0.240638  6.723   
38732 -0.373673 -0.859299 -0.931194  0.360702 -0.279134 -0.240638  7.263   
38733 -0.496356 -0.798870 -0.920724  0.361131 -0.248756 -0.587599  6.874   

         RMN   RMN

读取标准化数据之后，测试数据的反标准化是否正确。

In [17]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import joblib

# 加载标准化器

standardizer = DataStandardizer(columns= COLUMNS) # 指定需要标准化的列
# 加载标准化器
standardizer.load('scaler.joblib')

df_standardized = data

# 反标准化
df_original = standardizer.inverse_transform(df_standardized)

df_original

,BHC,CAL,DEN,DEPT,GR,HAC,RMG,RMN,RMN-RMG,SP,WellName
0,405.716,23.492,2.269,780.600,102.405,402.244,2.260,2.265,0.005,121.845,A1
1,404.701,23.453,2.274,780.650,103.093,397.115,2.241,2.281,0.040,121.845,A1
2,403.953,23.403,2.284,780.700,102.995,394.872,2.405,2.474,0.069,121.656,A1
3,404.434,23.363,2.274,780.750,102.405,397.009,2.598,2.640,0.042,121.325,A1
4,405.021,23.333,2.284,780.800,101.128,404.060,2.533,2.538,0.005,120.994,A1
...,...,...,...,...,...,...,...,...,...,...,...
38729,336.003,21.125,2.116,1006.079,81.215,335.470,6.191,8.237,2.046,66.313,A6
38730,333.760,21.095,2.144,1006.129,82.754,335.470,5.867,8.042,2.175,66.248,A6
38731,331.571,21.080,2.164,1006.179,84.678,335.150,6.723,9.164,2.441,66.150,A6
38732,329.755,21.118,2.178,1006.229,87.660,335.150,7.263,9.530,2.267,66.248,A6


5. 主程序

In [ ]:
# main.py

# from data_read import load_data_from_h5
# from data_process import DataStandardizer
# from data_save import save_data_to_h5
# from constants import COLUMNS

def main():
    # 加载数据
    filename = './well_log_daqing_standardized.h5'
    data, description = load_data_from_h5(filename)

    # 创建标准化器并拟合
    standardizer = DataStandardizer(columns=COLUMNS)
    standardizer.fit(data)

    # 标准化数据
    data_standardized = standardizer.transform(data)

    # 保存标准化后的数据
    file_description = "标准化后的数据"
    save_data_to_h5(data_standardized, 'well_log_daqing_standardized.h5', file_description)

if __name__ == "__main__":
    main()

## 代码设计

In [21]:
# data_loader.py

class DataLoaderStrategy:
    """
    数据加载策略的基类。
    """
    def load_data(self, filename):
        """
        加载数据的方法。
        """
        raise NotImplementedError("Subclasses must implement this method.")
    
class AdvancedDataLoader(DataLoaderStrategy):
    """
    高级的数据加载策略，使用其他方式加载数据。
    
    """
    def load_data(self, filename):
        # 实现高级的数据加载逻辑
        with h5py.File(filename, 'r') as h5f:
            file_description = h5f.attrs.get('file_description', 'No description')
            print(f"File Description: {file_description}")
            data_df = pd.DataFrame()
            for key in h5f.keys():
                dataset = h5f[key]
                data_df[key] = dataset[:]
                unit = dataset.attrs.get('unit', 'No unit')
                description = dataset.attrs.get('description', 'No description')
                print(f"{key}: Unit = {unit}, Description = {description}")
            data_df['WellName'] = data_df['WellName'].apply(lambda x: x.decode('utf-8'))
            return data_df
            
class DefaultDataLoader(DataLoaderStrategy):
    """
    默认的数据加载策略，使用data_read.py中的load_data_from_h5函数加载数据。
    """
    def load_data(self, filename):
        # 实现默认的数据加载逻辑
        with h5py.File(filename, 'r') as h5f:
            data = {key: h5f[key][()] for key in h5f.keys()}
            data_df = pd.DataFrame(data)
            data_df['WellName'] = data_df['WellName'].apply(lambda x: x.decode('utf-8'))
            return data_df
        
def load_data_from_h5(filename, strategy=DefaultDataLoader()):
    return strategy.load_data(filename)


# # 1. 默认数据加载策略
# # 导入必要的库
# import pandas as pd
# import h5py

# # 使用默认数据加载策略加载数据
# filename = "./well_log_daqing_standardized.h5"
# data_loader = DefaultDataLoader()
# df = load_data_from_h5(filename, strategy=data_loader)

# # 打印加载的数据
# print(df.head())

# # 1. 高级数据加载策略
# # 导入必要的库
# import pandas as pd
# import h5py

# # 使用高级数据加载策略加载数据
# filename = "./well_log_daqing.h5"
# data_loader = AdvancedDataLoader()
# df = load_data_from_h5(filename, strategy=data_loader)

# # 打印加载的数据和文件描述信息
# print(df.head())
# print(f"File Description: {file_description}")


In [ ]:
# data_saver.py

import h5py
import numpy as np
# from columns_description import COLUMN_DESCRIPTIONS
# columns_description.py
# 列描述
COLUMN_DESCRIPTIONS = {
    'DEPT': {
        'unit': 'm',
        'description': '深度，单位为米 (.M)'
    },
    'RMG': {
        'unit': 'ohmm',
        'description': '电阻率，单位为欧姆米 (.ohmm)'
    },
    'RMN': {
        'unit': 'ohmm',
        'description': '电阻率，单位为欧姆米 (.ohmm)'
    },
    'RMN-RMG': {
        'unit': 'ohmm',
        'description': '电阻率差值，没有明确单位，但由于是RMN与RMG的差值，其单位应该也是欧姆米 (.ohmm)'
    },
    'CAL': {
        'unit': 'cm',
        'description': '孔隙径，单位为厘米 (.cm)'
    },
    'SP': {
        'unit': 'mv',
        'description': '自发电位，单位为毫伏 (.mv)'
    },
    'GR': {
        'unit': 'API or unitless',
        'description': '伽马射线，单位未明确指出，但通常伽马射线的单位是API（美国石油学会单位）或者无单位'
    },
    'HAC': {
        'unit': 'us/m',
        'description': '声波时差，单位为微秒每米 (.us/m)'
    },
    'BHC': {
        'unit': 'us/m',
        'description': '表面声波时差，没有明确单位，可能是微秒每米 (.us/m)，因为与HAC类似'
    },
    'DEN': {
        'unit': 'g/cm3',
        'description': '密度，单位为克/立方厘米 (.g/cm3)'
    }
}


class DataSaveStrategy:
    """
    数据保存策略的基类。
    """
    def save_data(self, data, filename, file_description):
        """
        保存数据的方法。
        """
        raise NotImplementedError("Subclasses must implement this method.")
    
class AdvancedDataSaver(DataSaveStrategy):
    """
    高级的数据保存策
    """
    def save_data(self, data, filename, file_description):
        # 实现高级的数据保存逻辑
        with h5py.File(filename, 'w') as h5f:
            h5f.attrs['file_description'] = file_description
            for key in data.columns:
                dataset = h5f.create_dataset(key, data=data[key].values)
                if key in COLUMN_DESCRIPTIONS:
                    dataset.attrs['unit'] = COLUMN_DESCRIPTIONS[key]['unit']
                    dataset.attrs['description'] = COLUMN_DESCRIPTIONS[key]['description']
                    
class DefaultDataSaver(DataSaveStrategy):
    """
    默认的数据保存策略，使用data_save.py中的save_data_to_h5函数保存数据。
    """
    def save_data(self, data, filename, file_description):
        with h5py.File(filename, 'w') as h5f:
            h5f.attrs['file_description'] = file_description
            for key in data.columns:
                dataset = h5f.create_dataset(key, data=data[key].values)
                
def save_data_to_h5_using_strategy(data, filename, file_description, strategy=DefaultDataSaver()):
    strategy.save_data(data, filename, file_description)
    
# # 1. 默认数据保存策略
# # 导入必要的库
# import pandas as pd
# import h5py

# # 使用默认数据保存策略保存数据
# filename = "./well_log_daqing.h5"
# file_description = "这是一个示例文件。"
# data_saver = DefaultDataSaver()
# save_data_to_h5_using_strategy(df, filename, file_description, strategy=data_saver)
# print("数据已保存为well_log_daqing.h5。")

# # 2. 高级数据保存策略
# # 导入必要的库
# import pandas as pd
# import h5py

# # 使用高级数据加载策略加载数据
# filename = "./well_log_daqing.h5"
# file_description = "这是一个示例文件。"
# data_saver = AdvancedDataSaver()
# save_data_to_h5_using_strategy(df, filename, file_description, strategy=data_saver)
# print("数据已保存为well_log_daqing.h5。")


## 代码优化策略

### 加载数据优化，策略模式

使得两个数据加载函数适用于不同的场景，采用一些设计模式和技巧重构函数。常见的设计模式和技巧，如工厂模式，策略模式，参数化工厂模式，以及如何使用装饰器来扩展函数的功能

1. 工厂模式：一种创建对象的模式，提供了一种创建对象的方法，而不直接使用构造函数，这样可以为创建对象提供更多的灵活性。在这种情况下，可以创建一个工厂函数，根据输入参数选择不同的数据加载函数。
2. 策略模式：策略模式定义了一系列算法，并将每一个算法封装起来，使它们可以相互替换。策略模式使得算法可以独立于使用它的客户端。
3. 参数化工厂模式：参数化工厂模式是工厂模式的一种变体，允许客户端代码通过参数来指定所需的产品类型。
4. 装饰器扩展功能：装饰器是一种强大的技术，可以用来修改或增强函数的行为，而无需修改函数本身。例如，可以创建一个装饰器来添加日志记录功能。

通过设计模式和技巧，可以灵活地重构数据加载函数，使它们能够适应不同的场景和需求。选择哪种模式和技巧取决于具体需求和项目的复杂性。

场景：两个数据加载函数的主要区别在于它们如何处理数据集的加载和DataFrame的构建。一个函数直接从.h5文件中读取数据并返回。另一个是返回数据和文件描述信息。差异表明：数据加载函数可能需要处理不同的数据格式或需要提供不同级别的信息。

采用策略模式：允许在运行时选择不同的数据加载策略，这对于处理不同数据格式或提供不同级别的信息非常有用。此外，策略模式使得代码更加模块化和可扩展，可以轻松添加数据加载策略而不需要修改现有代码。

策略模式设计：

1. 建立抽象基类，定义了所以数据加载策略必须实现的接口。
2. 创建具体的数据加载策略类，分别实现不同的数据加载逻辑。
3. 创建一个上下文类或者函数，用于选择和执行不同的数据加载策略。

策略模式使用代码：

1. 编写执行实例
2. 使用上下文类或者函数选择和执行不同的数据加载策略。
3. 测试不同的数据加载策略。

### 保存数据优化，策略模式

策略模式设计数据保存的类，允许根据需要选择不同的数据保存策略来保存数据的类。
设计将根据选择不同的数据策略，例如：保存为.h5文件，保存为.csv文件，保存为数据库等。

策略模式设计：
1. 首先定义一个抽象基类，为每种保存策略创建具体的实现类。
2. 定义数据保存策略接口
3. 实现具体的数据保存策略
   1. 保存为.h5文件
   2. 保存为.csv文件
4. 使用数据保存策略
5. 执行实例

这种设计使得可以根据需要选择不同的数据保存策略，同时保持代码的清洗和模块化。通过传递不同的策略对象，可以轻松地改变数据保存的方式，而不需要修改sava_data_using_strategy函数的实现。

可以通过继承和重写某些方法来实现更灵活的数据保存策略。将创建一个基础的数据保存类，为不同的报错策略创建子类。

- 基础数据保存类：(注意，该基类的另外一种设计思路)
- 具体的保存策略
  - 保存每列数据为单独的数据集
  - 保存整个DataFrame为单个数据集

```python 
import h5py
import pandas as pd

class BaseH5DataSaver:
    def __init__(self, filename, file_description=None):
        self.filename = filename
        self.file_description = file_description

    def save_data(self, data):
        with h5py.File(self.filename, 'w') as h5f:
            if self.file_description:
                h5f.attrs['file_description'] = self.file_description
            self._save_data_to_h5(data, h5f)

    def _save_data_to_h5(self, data, h5f):
        raise NotImplementedError("Subclasses must implement this method")
```